# A Baseline Implementation for SE125 Project 2

We provide a baseline model for conversation modeling using deep learning.


## 1. Libraries
In this section, we import third-party libraries to be used in this project.
You may need to install them using `pip`:
```
    pip install tqdm
    pip install cython
    pip install tables
    pip install tensorboardX
    ...
```

In [1]:
import numpy as np
import time
import os
import math
import sys
import tables
import json
import random
from tqdm import tqdm

import torch 
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG, format="%(message)s")#,format="%(asctime)s: %(name)s: %(levelname)s: %(message)s")


## 2. Utilities

In this section we maintain utilities for model construction and training. 
Please put your own utility modules/functions in this section.

In [2]:
PAD_ID, SOS_ID, EOS_ID, UNK_ID = [0, 1, 2, 3]

def asHHMMSS(s):
    m = math.floor(s / 60)
    s -= m * 60
    h = math.floor(m /60)
    m -= h *60
    return '%d:%d:%d'% (h, m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s<%s'%(asHHMMSS(s), asHHMMSS(rs))

#######################################################################
import nltk
try: 
    nltk.word_tokenize("hello world")
except LookupError: 
    nltk.download('punkt')
    
def sent2indexes(sentence, vocab, maxlen):
    '''sentence: a string or list of string
       return: a numpy array of word indices
    '''      
    def convert_sent(sent, vocab, maxlen):
        idxes = np.zeros(maxlen, dtype=np.int64)
        idxes.fill(PAD_ID)
        tokens = nltk.word_tokenize(sent.strip())
        idx_len = min(len(tokens), maxlen)
        for i in range(idx_len): idxes[i] = vocab.get(tokens[i], UNK_ID)
        return idxes, idx_len
    if type(sentence) is list:
        inds, lens = [], []
        for sent in sentence:
            idxes, idx_len = convert_sent(sent, vocab, maxlen)
            #idxes, idx_len = np.expand_dims(idxes, 0), np.array([idx_len])
            inds.append(idxes)
            lens.append(idx_len)
        return np.vstack(inds), np.vstack(lens)
    else:
        inds, lens = sent2indexes([sentence], vocab, maxlen)
        return inds[0], lens[0]

def indexes2sent(indexes, vocab, ignore_tok=PAD_ID): 
    '''indexes: numpy array'''
    def revert_sent(indexes, ivocab, ignore_tok=PAD_ID):
        toks=[]
        length=0
        indexes=filter(lambda i: i!=ignore_tok, indexes)
        for idx in indexes:
            toks.append(ivocab[idx])
            length+=1
            if idx == EOS_ID:
                break
        return ' '.join(toks), length
    
    ivocab = {v: k for k, v in vocab.items()}
    if indexes.ndim==1:# one sentence
        return revert_sent(indexes, ivocab, ignore_tok)
    else:# dim>1
        sentences=[] # a batch of sentences
        lens=[]
        for inds in indexes:
            sentence, length = revert_sent(inds, ivocab, ignore_tok)
            sentences.append(sentence)
            lens.append(length)
        return sentences, lens
    
def save_model(model, epoch):
    """Save model parameters to checkpoint"""
    ckpt_path=f'./output/checkpoint_iter{epoch}.pkl'
    #print(f'Saving model parameters to {ckpt_path}')
    torch.save(model.state_dict(), ckpt_path)
        
def load_model(model, epoch):
    """Load parameters from checkpoint"""
    ckpt_path=f'./output/checkpoint_iter{epoch}.pkl'
    #print(f'Loading model parameters from {ckpt_path}')
    model.load_state_dict(torch.load(ckpt_path))

## 3. Configuration
In this section, we configurate some hyperparameters for the model.

In [3]:
def get_config():
    conf = {
    'maxlen':40, # maximum utterance length
    'diaglen':10, # how many utterance kept in the context window

    # Model Arguments
    'emb_size':200, # size of word embeddings
    'rnn_hid_utt':512, # number of rnn hidden units for utterance encoder
    'rnn_hid_ctx':512, # number of rnn hidden units for context encoder
    'rnn_hid_dec':512, # number of rnn hidden units for decoder
    # 'n_layers':1, # number of layers
    'dropout':0.5, # dropout applied to layers (0 = no dropout)
    'teach_force': 0.8, # use teach force for decoder
      
    # Training Arguments
    'batch_size':64,
    'epochs':10, # maximum number of epochs
    'lr':2e-4, # autoencoder learning rate
    'beta1':0.9, # beta1 for adam
    'init_w':0.05, # initial w
    'clip':5.0,  # gradient clipping, max norm


    'd_model' : 512,  # Embedding Size
    'd_ff': 2048, # FeedForward dimension
    'd_k' : 64,
    'd_v':64, # dimension of K(=Q), V
    'n_layers' : 6,  # number of Encoder of Decoder Layer
    'n_heads' : 8  # number of heads in Multi-Head Attention
    }
    return conf 

## 4. Data Loader
A tool to load batches from the binarized (.h5) dataset

In [4]:
class DialogDataset(data.Dataset):
    def __init__(self, filepath, max_ctx_len=7, max_utt_len=40):
        # 1. Initialize file path or list of file names.
        """read training sentences(list of int array) from a hdf5 file"""
        self.max_ctx_len=max_ctx_len
        self.max_utt_len=max_utt_len
        
        print("loading data...")
        table = tables.open_file(filepath)
        self.data = table.get_node('/sentences')[:].astype(np.long)
        self.index = table.get_node('/indices')[:]
        self.data_len = self.index.shape[0]
        print("{} entries".format(self.data_len))

    def __getitem__(self, offset):
        pos_utt, ctx_len, res_len = self.index[offset]['pos_utt'], self.index[offset]['ctx_len'], self.index[offset]['res_len']
        ctx_arr=self.data[pos_utt-ctx_len:pos_utt]
        res_arr=self.data[pos_utt:pos_utt+res_len]
        ## split context array into utterances
        context=[]
        utt_lens=[]
        utt=[]
        for i, tok in enumerate(ctx_arr):
            utt.append(ctx_arr[i])
            if tok==EOS_ID:
                if len(utt)<self.max_utt_len+1:
                    utt_lens.append(len(utt)-1)# floor is not counted in the utt length
                    utt.extend([PAD_ID]*(self.max_utt_len+1-len(utt)))  
                else:
                    utt=utt[:self.max_utt_len+1]
                    utt[-1]=EOS_ID
                    utt_lens.append(self.max_utt_len)
                context.append(utt)                
                utt=[]    
        if len(context)>self.max_ctx_len: # trunk long context
            context=context[-self.max_ctx_len:]
            utt_lens=utt_lens[-self.max_ctx_len:]
        context_len=len(context)
        
        if len(context)<self.max_ctx_len: # pad short context
            for i in range(len(context), self.max_ctx_len):
                context.append([0, SOS_ID, EOS_ID]+[PAD_ID]*(self.max_utt_len-2)) # [floor, <sos>, <eos>, <pad>, <pad> ...]
                utt_lens.append(2) # <s> and </s>
        context = np.array(context)        
        utt_lens=np.array(utt_lens)
        floors=context[:,0]
        context = context[:,1:]
        
        ## Padding ##    
        response = res_arr[1:]
        if len(response)<self.max_utt_len:
            res_len=len(response)
            response=np.append(response,[PAD_ID]*(self.max_utt_len-len(response)))
        else:
            response=response[:self.max_utt_len]
            response[-1]=EOS_ID
            res_len=self.max_utt_len

        return context, context_len, utt_lens, floors, response, res_len

    def __len__(self):
        return self.data_len
    

def load_dict(filename):
    return json.loads(open(filename, "r").readline())

def load_vecs(fin):         
    """read vectors (2D numpy array) from a hdf5 file"""
    h5f = tables.open_file(fin)
    h5vecs= h5f.root.vecs
    
    vecs=np.zeros(shape=h5vecs.shape,dtype=h5vecs.dtype)
    vecs[:]=h5vecs[:]
    h5f.close()
    return vecs

## 5. Models
Define your model(including its dependent sub-modules) here. 

### 5.1 Positional Encoding
由于 Transformer 模型没有循环神经网络的迭代操作，所以我们必须提供每个字的位置信息给 Transformer，这样它才能识别出语言中的顺序关系
位置嵌入的维度为 ( max_sequence_length, embedding_dimension ), 位置嵌入的维度与词向量的维度是相同的，都是 embedding_dimension
使用 sin 和 cos 函数的线性变换来提供给模型位置信息:
$$ PE{(pos,2i)} = sin(pos / 10000^{2i/d_{\text{model}}}) \\ PE{(pos,2i+1)} = cos(pos / 10000^{2i/d_{\text{model}}}) $$

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        '''
        x: [seq_len, batch_size, d_model]
        '''
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

### 5.2 Pad Mask
因为每个批次输入序列长度是不一样的也就是说，我们要对输入序列进行对齐。
Self Attention 的计算过程中，我们通常使用 mini-batch 来计算，也就是一次计算多句话，即 $$ X $$ 的维度是 [batch_size, sequence_length]，$$ sequence\_ length $$ 是句长。
而一个 mini-batch 是由多个不等长的句子组成的，我们需要按照这个 mini-batch 中最大的句长对剩余的句子进行补齐，一般用 0 进行填充，这个过程叫做 padding

但这时在进行 softmax 就会产生问题。回顾 softmax 函数 $$ \sigma (z_i)=\frac {e^{z_i}}{\sum_{j=1}^K e^{z_j}} $$ ， $$ e^0 $$是 1，是有值的，这样的话 softmax 中被 padding 的部分就参与了运算，相当于让无效的部分参与了运算，这可能会产生很大的隐患。因此需要做一个 mask 操作，让这些无效的区域不参与运算，一般是给无效区域加一个很大的负数偏置，即:
\begin{align*} &Z_{illegal}=Z_{illegal}+bias_{illegal}\\
&bias_{illegal}→-∞
\end{align*}

In [ ]:
def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token

    #返回一个大小和 seq_k 一样的 tensor，值只有 True 和 False
    # 如果 seq_k 某个位置的值等于 0，那么对应位置就是 True，否则即为 False
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], False is masked
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]

### 5.3 Subsequence Mask
sequence mask 是为了使得 decoder 不能看见未来的信息。也就是对于一个序列，在 time_step 为 t 的时刻，我们的解码输出应该只能依赖于 t 时刻之前的输出，而不能依赖 t 之后的输出

In [ ]:
def get_attn_subsequence_mask(seq):
    '''
    seq: [batch_size, tgt_len]
    '''

    # 首先通过 np.ones() 生成一个全 1 的方阵，然后通过 np.triu() 生成一个上三角矩阵
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    subsequence_mask = np.triu(np.ones(attn_shape), k=1) # Upper triangular matrix
    subsequence_mask = torch.from_numpy(subsequence_mask).byte()
    return subsequence_mask # [batch_size, tgt_len, tgt_len]

### 5.4 ScaledDotProductAttention

K、Q、V 分别代表什么:

* 在 encoder 的 self-attention 中，Q、K、V 都来自同一个地方，它们是上一层 encoder 的输出。对于第一层 encoder，它们就是 word embedding 和 positional encoding 相加得到的输入。
* 在 decoder 的 self-attention 中，Q、K、V 也是自于同一个地方，它们是上一层 decoder 的输出。对于第一层 decoder，同样也是 word embedding 和 positional encoding 相加得到的输入。但是对于 decoder，我们不希望它能获得下一个 time step (即将来的信息，不想让他看到它要预测的信息)，因此我们需要进行 sequence masking。
* 在 encoder-decoder attention 中，Q 来自于 decoder 的上一层的输出，K 和 V 来自于 encoder 的输出，K 和 V 是一样的。
Q、K、V 的维度都是一样的

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        '''
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        '''
        '''
        相乘之后得到的 scores 还不能立刻进行 softmax，需要和 attn_mask 相加，把一些需要屏蔽的信息屏蔽掉
        '''
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(64) # scores : [batch_size, n_heads, len_q, len_k]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is True.

        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V) # [batch_size, n_heads, len_q, d_v]
        return context, attn

### 5.6 MultiHeadAttention
前面定义的一组$$ Q,K,V $$  可以让一个词 attend to 相关的词，我们可以定义多组 $$ Q,K,V $$，让它们分别关注不同的上下文
计算$$ Q,K,V $$的过程还是一样，只不过线性变换的矩阵从一组$$ (W^Q,W^K,W^V) $$  变成了多组 $$ (W^Q_0,W^K_0,W^V_0) $$ , $$ (W^Q_1,W^K_1,W^V_1) $$   ，

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self,config):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(config['d_model'], config['d_k'] *config['n_heads'], bias=False)
        self.W_K = nn.Linear(config['d_model'], config['d_k'] * config['n_heads'], bias=False)
        self.W_V = nn.Linear(config['d_model'], config['d_v'] * config['n_heads'], bias=False)
        self.fc = nn.Linear(config['n_heads'] * config['d_v'], config['d_model'], bias=False)
    def forward(self,config, input_Q, input_K, input_V, attn_mask):
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        residual, batch_size = input_Q, input_Q.size(0)
        # (B, S, D) -proj-> (B, S, D_new) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        Q = self.W_Q(input_Q).view(batch_size, -1, config['n_heads'], config['d_k']).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, config['n_heads'], config['d_k']).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, config['n_heads'], config['d_v']).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, config['n_heads'], 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]

        # context: [batch_size, n_heads, len_q, d_v], attn: [batch_size, n_heads, len_q, len_k]
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
        context = context.transpose(1, 2).reshape(batch_size, -1, config['n_heads'] *config[' d_v']) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc(context) # [batch_size, len_q, d_model]
        return nn.LayerNorm(config['d_model']).cuda()(output + residual), attn

### 5.7 FeedForward Layer

In [ ]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self,config):
        super(PoswiseFeedForwardNet, self).__init__()
        '''
            做两次线性变换，残差连接后再跟一个 Layer Norm
        '''
        self.fc = nn.Sequential(

            nn.Linear(config['d_model'], config['d_ff'], bias=False),
            nn.ReLU(),
            nn.Linear(config['d_ff'], config['d_model'], bias=False)
        )
    def forward(self, config,inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(config['d_model']).cuda()(output + residual) # [batch_size, seq_len, d_model]

### 5.9 Encoder Layer
Encoder block 的计算过程:
1. 字向量与位置编码
$$ X = Embedding\ Lookup(X) + Positional\ Encoding $$
2.  自注意力机制
$$
Q = Linear(X) = XW_{Q}\\
K = Linear(X) = XW_{K}\\
V = Linear(X) = XW_{V}\\
X_{attention} = SelfAttention(Q, \ K, \ V)
$$

3.  self-attention 残差连接与 Layer Normalization
$$ X_{attention} = X + X_{attention} \\ $$
$$ X_{attention} = LayerNorm(X_{attention}) $$

4.  FeedForward
两层线性映射并用激活函数激活

5.  FeedForward 残差连接与 Layer Normalization
$$ X_{hidden} = X_{attention} + X_{hidden}\\ $$
$$ X_{hidden} = LayerNorm(X_{hidden}) $$


In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self,config):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention(config)
        self.pos_ffn = PoswiseFeedForwardNet(config)

    def forward(self, enc_inputs, enc_self_attn_mask):
        '''
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]
        '''
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn

### 5.10 Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self,config,vocab_size):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(vocab_size, config['d_model'])
        self.pos_emb = PositionalEncoding(config['d_model'])
        # 使用 nn.ModuleList() 里面的参数是列表，列表里面存了 n_layers 个 Encoder Layer
        self.layers = nn.ModuleList([EncoderLayer(config) for _ in range(config['n_layers'])])

    def forward(self, enc_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs) # [batch_size, src_len, src_len]
        enc_self_attns = []
        for layer in self.layers:
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

### 5.9 Decoder Layer
Decoder 结构，从下到上依次是：
 * Masked Multi-Head Self-Attention
 * Multi-Head Encoder-Decoder Attention
 * FeedForward Network
和 Encoder 一样，上面三个部分的每一个部分，都有一个残差连接，后接一个 Layer Normalization。

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self,config):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention(config)
        self.dec_enc_attn = MultiHeadAttention(config)
        self.pos_ffn = PoswiseFeedForwardNet(config)

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        '''
        dec_inputs: [batch_size, tgt_len, d_model]
        enc_outputs: [batch_size, src_len, d_model]
        dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
        dec_enc_attn_mask: [batch_size, tgt_len, src_len]
        '''
        # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        # dec_outputs: [batch_size, tgt_len, d_model], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_outputs = self.pos_ffn(dec_outputs) # [batch_size, tgt_len, d_model]
        return dec_outputs, dec_self_attn, dec_enc_attn

### 5.10 Decoder

In [ ]:
class Decoder(nn.Module):
    def __init__(self,config,vocab_size):
        super(Decoder, self).__init__()
        self.tgt_emb = nn.Embedding(vocab_size, config['d_model'])
        self.pos_emb = PositionalEncoding(config['d_model'])
        self.layers = nn.ModuleList([DecoderLayer(config) for _ in range(config['n_layers'])])

    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        '''
        dec_inputs: [batch_size, tgt_len]
        enc_intpus: [batch_size, src_len]
        enc_outputs: [batsh_size, src_len, d_model]
        '''
        dec_outputs = self.tgt_emb(dec_inputs) # [batch_size, tgt_len, d_model]
        dec_outputs = self.pos_emb(dec_outputs.transpose(0, 1)).transpose(0, 1).cuda() # [batch_size, tgt_len, d_model]

        '''
        Decoder 中不仅要把 "pad"mask 掉，还要 mask 未来时刻的信息
        torch.gt(a, value) 的意思是，将 a 中各个位置上的元素和 value 比较，若大于 value，则该位置取 1，否则取 0
        '''
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        dec_self_attn_subsequence_mask = get_attn_subsequence_mask(dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequence_mask), 0).cuda() # [batch_size, tgt_len, tgt_len]

        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs) # [batc_size, tgt_len, src_len]

        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        return dec_outputs, dec_self_attns, dec_enc_attns

In [5]:
class MyModel(nn.Module):
    '''The basic Hierarchical Recurrent Encoder-Decoder model. '''
    # def __init__(self, config, vocab_size):
    #     super(MyModel, self).__init__()
    #     self.vocab_size = vocab_size
    #     self.maxlen=config['maxlen']
    #     self.clip = config['clip']
    #     self.init_w = config['init_w']
    #
    #     self.embedder= nn.Embedding(vocab_size, config['emb_size'], padding_idx=PAD_ID)
    #     self.utt_encoder = RNNEncoder(self.embedder, config['emb_size'], config['rnn_hid_utt'], True,
    #                                config['n_layers'], config['dropout'])
    #                                                     # utter encoder: encode response to vector
    #     self.context_encoder = ContextEncoder(self.utt_encoder, config['rnn_hid_utt']*2,
    #                                           config['rnn_hid_ctx'], 1, config['dropout'])
    #                                           # context encoder: encode context to vector
    #     self.decoder = RNNDecoder(self.embedder, config['emb_size'], config['rnn_hid_ctx'], vocab_size, 1, config['dropout']) # utter decoder: P(x|c,z)
    #     self.optimizer = optim.Adam(list(self.context_encoder.parameters())
    #                                   +list(self.decoder.parameters()),lr=config['lr'])
    def __init__(self,config,vocab_size):
        super(MyModel, self).__init__()
        self.encoder = Encoder(config,vocab_size).cuda()
        self.decoder = Decoder(config,vocab_size).cuda()
        self.projection = nn.Linear(config['d_model'], vocab_size, bias=False).cuda()

    def forward(self, enc_inputs, context_lens,utt_lens,dec_inputs,res_lens):
        '''
        enc_inputs: [batch_size, src_len]
        dec_inputs: [batch_size, tgt_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]

        criterion = nn.CrossEntropyLoss(ignore_index=0)


        outputs=dec_logits.view(-1, dec_logits.size(-1))
        loss = criterion(outputs, dec_outputs.view(-1))
        return loss

    def train_batch(self, context, context_lens, utt_lens, response, res_lens):
        # self.encoder.train()
        # self.decoder.train()


        loss = self.forward(context, context_lens, utt_lens, response, res_lens)
        
        self.optimizer.zero_grad()
        loss.backward()
        # `clip_grad_norm` to prevent exploding gradient in RNNs
        nn.utils.clip_grad_norm_(list(self.context.parameters())+list(self.decoder.parameters()), self.clip)
        self.optimizer.step()
        
        return {'train_loss': loss.item()}      
    
    def valid(self, context, context_lens, utt_lens, response, res_lens):
        self.context.eval()
        self.decoder.eval()        
        loss = self.forward(context, context_lens, utt_lens, response, res_lens)
        return {'valid_loss': loss.item()}
    
    def sample(self, context, context_lens, utt_lens, n_samples):    
        self.context.eval()
        self.decoder.eval()
        with torch.no_grad():
            c = self.encoder(context, context_lens, utt_lens)
        sample_words, sample_lens = self.decoder.sampling(c, None, None, None, n_samples, self.maxlen)  
        return sample_words, sample_lens  

## 6. Evaluation
We provide the evaluation script as well as the BLEU score metric. 

**Do not change code in this block**

In [6]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from collections import Counter

class Metrics:
    """
    """
    def __init__(self):
        super(Metrics, self).__init__()

    def sim_bleu(self, hyps, ref):
        """
        :param ref - a list of tokens of the reference
        :param hyps - a list of tokens of the hypothesis
    
        :return maxbleu - recall bleu
        :return avgbleu - precision bleu
        """
        scores = []
        for hyp in hyps:
            try:
                scores.append(sentence_bleu([ref], hyp, smoothing_function=SmoothingFunction().method7,
                                        weights=[1./4, 1./4, 1./4, 1./4]))
            except:
                scores.append(0.0)
        return np.max(scores), np.mean(scores)
    
def evaluate(model, metrics, test_loader, vocab, repeat, f_eval):
    ivocab = {v: k for k, v in vocab.items()}
    device = next(model.parameters()).device
    
    recall_bleus, prec_bleus, avg_lens  = [], [], []
        
    dlg_id = 0
    for context, context_lens, utt_lens, floors, response, res_lens in tqdm(test_loader): 
        
        if dlg_id > 5000: break
        
#        max_ctx_len = max(context_lens)
        max_ctx_len = context.size(1)
        context, utt_lens, floors = context[:,:max_ctx_len,1:], utt_lens[:,:max_ctx_len]-1, floors[:,:max_ctx_len] 
                         # remove empty utts and the sos token in the context and reduce the context length
        ctx, ctx_lens = context, context_lens
        context, context_lens, utt_lens \
            = [tensor.to(device) for tensor in [context, context_lens, utt_lens]]

#################################################
        utt_lens[utt_lens<=0]=1
#################################################
        
        with torch.no_grad():
            sample_words, sample_lens = model.sample(context, context_lens, utt_lens, repeat)
        # nparray: [repeat x seq_len]       
        
        pred_sents, _ = indexes2sent(sample_words, vocab)
        pred_tokens = [sent.split(' ') for sent in pred_sents]   
        ref_str, _ =indexes2sent(response[0].numpy(), vocab, SOS_ID)
        #ref_str = ref_str.encode('utf-8')
        ref_tokens = ref_str.split(' ')
        
        max_bleu, avg_bleu = metrics.sim_bleu(pred_tokens, ref_tokens)
        recall_bleus.append(max_bleu)
        prec_bleus.append(avg_bleu)
        
        avg_lens.append(np.mean(sample_lens))

        response, res_lens = [tensor.to(device) for tensor in [response, res_lens]]
        
        ## Write concrete results to a text file
        dlg_id += 1 
        if f_eval is not None:
            f_eval.write("Batch {:d} \n".format(dlg_id))
            # print the context
            start = np.maximum(0, ctx_lens[0]-5)
            for t_id in range(start, ctx_lens[0], 1):
                context_str = indexes2sent(ctx[0, t_id].numpy(), vocab)
                f_eval.write("Context {:d}-{:d}: {}\n".format(t_id, floors[0, t_id], context_str))
            #print the ground truth response    
            f_eval.write("Target >> {}\n".format(ref_str.replace(" ' ", "'")))
            for res_id, pred_sent in enumerate(pred_sents):
                f_eval.write("Sample {:d} >> {}\n".format(res_id, pred_sent.replace(" ' ", "'")))
            f_eval.write("\n")
    prec_bleu= float(np.mean(prec_bleus))
    recall_bleu = float(np.mean(recall_bleus))
    result = {'avg_len':float(np.mean(avg_lens)),
              'recall_bleu': recall_bleu, 'prec_bleu': prec_bleu, 
              'f1_bleu': 2*(prec_bleu*recall_bleu) / (prec_bleu+recall_bleu+10e-12),
             }
    
    if f_eval is not None:
        for k, v in result.items():
            f_eval.write(str(k) + ':'+ str(v)+' ')
        f_eval.write('\n')
    print("Done testing")
    print(result)
    
    return result


## 7. Training
The training script here.

In [7]:
import argparse
from datetime import datetime
from tensorboardX import SummaryWriter # install tensorboardX (pip install tensorboardX) before importing this package

def train(args, model=None, pad = 0):
    # LOG #
    fh = logging.FileHandler(f"./output/logs.txt")
                                      # create file handler which logs even debug messages
    logger.addHandler(fh)# add the handlers to the logger
    
    timestamp = datetime.now().strftime('%Y%m%d%H%M')
    tb_writer = SummaryWriter(f"./output/logs/{timestamp}") if args.visual else None

    # Set the random seed manually for reproducibility.
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    device = torch.device(f"cuda:{args.gpu_id}" if torch.cuda.is_available() else "cpu")
    print(device)


    config=get_config()

    if args.visual:
        json.dump(config, open(f'./output/config_{timestamp}.json', 'w'))# save configs

    ###############################################################################
    # Load data
    ###############################################################################
    data_path = args.data_path+args.dataset+'/'
    train_set = DialogDataset(os.path.join(data_path, 'train.h5'), config['diaglen'], config['maxlen'])
    valid_set = DialogDataset(os.path.join(data_path, 'valid.h5'), config['diaglen'], config['maxlen'])
    test_set = DialogDataset(os.path.join(data_path, 'test.h5'), config['diaglen'], config['maxlen'])
    vocab = load_dict(os.path.join(data_path, 'vocab.json'))
    ivocab = {v: k for k, v in vocab.items()}
    n_tokens = len(ivocab)
    metrics=Metrics()    
    print("Loaded data!")

    ###############################################################################
    # Define the models
    ###############################################################################
    if model is None:
        model = MyModel(config, n_tokens)

    if args.reload_from>=0:
        load_model(model, args.reload_from)
        
    model=model.to(device)

    logger.info("Training...")
    best_perf = -1
    itr_global=1
    start_epoch=1 if args.reload_from==-1 else args.reload_from+1
    for epoch in range(start_epoch, config['epochs']+1):
        epoch_start_time = time.time()
        itr_start_time = time.time()
        
        # shuffle (re-define) data between epochs   
        train_loader=torch.utils.data.DataLoader(dataset=train_set, batch_size=config['batch_size'],
                                                 shuffle=True, num_workers=1, drop_last=True)
        n_iters=train_loader.__len__()
        itr = 1
        for batch in train_loader:# loop through all batches in training data
            model.train()
            context, context_lens, utt_lens, floors, response, res_lens = batch

 #           max_ctx_len = max(context_lens)
            max_ctx_len = context.size(1)
            context, utt_lens = context[:,:max_ctx_len,1:], utt_lens[:,:max_ctx_len]-1
                                    # remove empty utterances in context
                                    # remove the sos token in the context and reduce the context length     
#################################################
            utt_lens[utt_lens<=0]=1
#################################################
            batch_gpu = [tensor.to(device) for tensor in [context, context_lens, utt_lens, response, res_lens]] 
            train_results = model.train_batch(*batch_gpu)
                     
            if itr % args.log_every == 0:
                elapsed = time.time() - itr_start_time
                log = '%s|%s@gpu%d epo:[%d/%d] iter:[%d/%d] step_time:%ds elapsed:%s'\
                %(args.model, args.dataset, args.gpu_id, epoch, config['epochs'],
                         itr, n_iters, elapsed, timeSince(epoch_start_time,itr/n_iters))
                logger.info(log)
                logger.info(train_results)
                if args.visual:
                    tb_writer.add_scalar('train_loss', train_results['train_loss'], itr_global)

                itr_start_time = time.time()    
                
            if itr % args.valid_every == 0 and False:
                logger.info('Validation ')
                valid_loader=torch.utils.data.DataLoader(dataset=valid_set, batch_size=config['batch_size'], shuffle=True, num_workers=1)
                model.eval()    
                valid_losses = []
                for context, context_lens, utt_lens, floors, response, res_lens in valid_loader:
 #                   max_ctx_len = max(context_lens)
                    max_ctx_len = context.size(1)
                    context, utt_lens = context[:,:max_ctx_len,1:], utt_lens[:,:max_ctx_len]-1
                             # remove empty utterances in context
                             # remove the sos token in the context and reduce the context length
#################################################
                    utt_lens[utt_lens<=0]=1
#################################################
                    batch = [tensor.to(device) for tensor in [context, context_lens, utt_lens, response, res_lens]]
                    valid_results = model.valid(*batch)    
                    valid_losses.append(valid_results['valid_loss'])
                if args.visual: tb_writer.add_scalar('valid_loss', np.mean(valid_losses), itr_global)
                logger.info({'valid_loss':np.mean(valid_losses)})    
                
            itr += 1
            itr_global+=1            
            
            if itr_global % args.eval_every == 0:  # evaluate the model in the validation set
                model.eval()          
                logger.info("Evaluating in the validation set..")

                valid_loader=torch.utils.data.DataLoader(dataset=valid_set, batch_size=1, shuffle=False, num_workers=1)

                f_eval = open(f"./output/tmp_results/iter{itr_global}.txt", "w")
                repeat = 10            
                eval_results = evaluate(model, metrics, valid_loader, vocab, repeat, f_eval)
                bleu = eval_results['recall_bleu']
                if bleu> best_perf:
                    save_model(model, 0)#itr_global) # save model after each epoch
                if args.visual:
                    tb_writer.add_scalar('recall_bleu', bleu, itr_global)
                
        # end of epoch ----------------------------
               # model.adjust_lr()

    return model


## 8. Main Function (Training)
You can change the default arguments by setting the `default` attribute.

In [ ]:

if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='Dialog Pytorch')
    # Path Arguments
    parser.add_argument('--data_path', type=str, default='./data/', help='location of the data corpus')
    parser.add_argument('--model', type=str, default='MyModel', help='model name')
    parser.add_argument('--dataset', type=str, default='weibo', help='name of dataset.')
    parser.add_argument('-v','--visual', action='store_true', default=False, help='visualize training status in tensorboard')
    parser.add_argument('--reload_from', type=int, default=-1, help='reload from a trained ephoch')
    parser.add_argument('--gpu_id', type=int, default=0, help='GPU ID')

    # Evaluation Arguments
    parser.add_argument('--log_every', type=int, default=100, help='interval to log autoencoder training results')
    parser.add_argument('--valid_every', type=int, default=1000, help='interval to validation')
    parser.add_argument('--eval_every', type=int, default=2000, help='interval to evaluation to concrete results')
    parser.add_argument('--seed', type=int, default=1111, help='random seed')
    
    
    
    
    args = parser.parse_args(args=[])
    print(vars(args))

    # make output directory if it doesn't already exist
    os.makedirs(f'./output/models', exist_ok=True)
    os.makedirs(f'./output/tmp_results', exist_ok=True)
        
    torch.backends.cudnn.benchmark = True # speed up training by using cudnn
    torch.backends.cudnn.deterministic = True # fix the random seed in cudnn
    
    model = train(args)

{'data_path': './data/', 'model': 'MyModel', 'dataset': 'weibo', 'visual': False, 'reload_from': -1, 'gpu_id': 0, 'log_every': 100, 'valid_every': 1000, 'eval_every': 2000, 'seed': 1111}
cpu
loading data...
15481891 entries
loading data...
89994 entries
loading data...
89994 entries
Loaded data!


Training...


## 9. Main Function (Test)

**Please do not change code here except the default arguments**

In [ ]:

def test(args):
    conf = get_config()
    # Set the random seed manually for reproducibility.
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(args.seed)
    else:
        print("Note that our pre-trained models require CUDA to evaluate.")
    
    # Load data
    data_path=args.data_path+args.dataset+'/'
    test_set=DialogDataset(data_path+'test.h5', conf['diaglen'], conf['maxlen'])
    test_loader=torch.utils.data.DataLoader(dataset=test_set, batch_size=1, shuffle=False, num_workers=1)
    vocab = load_dict(data_path+'vocab.json')
    n_tokens = len(vocab)

    metrics=Metrics()
    
    # Load model checkpoints    
    model = MyModel(conf, n_tokens)
    load_model(model, 0)
    #model=model.to(device)
    model.eval()
    
    f_eval = open("./output/results.txt", "w")
    repeat = args.n_samples
    
    evaluate(model, metrics, test_loader, vocab, repeat, f_eval)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='PyTorch DialogGAN for Eval')
    parser.add_argument('--data_path', type=str, default='./data/', help='location of the data corpus')
    parser.add_argument('--dataset', type=str, default='weibo', help='name of dataset, SWDA or DailyDial')
    parser.add_argument('--model', type=str, default='MyModel', help='model name')
    parser.add_argument('--reload_from', type=int, default=0, 
                        help='directory to load models from')
    
    parser.add_argument('--n_samples', type=int, default=10, help='Number of responses to sampling')
    parser.add_argument('--seed', type=int, default=1111, help='random seed')
    args = parser.parse_args(args=[])
    print(vars(args))
    test(args)